# Simulating a rock sample

Notebook to generate a set of spheres

#### Import libraries

In [1]:
from IPython.display import Markdown as md
from IPython.display import display as dp
import string as st
import sys
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle
import datetime

from fatiando.utils import ang2vec, vec2ang
from fatiando.mesher import Sphere, Prism, PolygonalPrism
from fatiando.gravmag import sphere, prism, polyprism

/home/andrelreis/anaconda2/lib/python2.7/site-packages/fatiando/vis/mpl.py:76: UserWarning: This module will be removed in v0.6. We recommend the use of matplotlib.pyplot module directly. Some of the fatiando specific functions will remain.
  "specific functions will remain.")


In [2]:
notebook_name = 'simulation_rock_sample.ipynb'

#### Importing auxiliary functions

In [3]:
dir_modules = '../../'
sys.path.append(dir_modules)

In [4]:
import my_package as fc

#### Loading 2D grid properties

In [5]:
with open('data/regular_grid.pickle') as f:
        regular = pickle.load(f)

## Open a dictionary

In [6]:
model_rock = dict()

#### List of saved files

In [7]:
saved_files = []

### Observation area

In [8]:
print 'Area limits: \n x_max = %.1f $\mu m$ \n x_min = %.1f $\mu m$ \n y_max = %.1f $\mu m$ \n y_min = %.1f $\mu m$' % (regular['area'][1], regular['area'][0],regular['area'][3],regular['area'][2])

Area limits: 
 x_max = 15000.0 $\mu m$ 
 x_min = -15000.0 $\mu m$ 
 y_max = 15000.0 $\mu m$ 
 y_min = -15000.0 $\mu m$


## Creating a simple model rock sample  

### Side lengths

In [9]:
model_rock['lx'] = 18000.
model_rock['ly'] = 12000.
model_rock['lz'] = 2000. 

### Magnetization direction

In [10]:
mag = 1.5
inc = 20.
dec = 30.

In [11]:
model_rock['mag_prism']= ang2vec(mag,inc,dec)

### Generating model

In [12]:
model_rock['prism'] = [Prism(-0.5*model_rock['lx'], 0.5*model_rock['lx'],
                            -0.5*model_rock['ly'], 0.5*model_rock['ly'],
                            -0.5*model_rock['lz'], 0.5*model_rock['lz'],
                            {'magnetization':model_rock['mag_prism']})]

### Random spheres within prism

#### Generating the center coordinates and dimension of spheres

In [13]:
np.random.seed(seed=40)
n = 1000
R = 60.
dec_min = 0.
dec_max = 3.
inc_min = 88.
inc_max = 90.
Coordx = np.random.uniform(-0.5*model_rock['lx']+R,+0.5*model_rock['lx']-R,n) 
Coordy = np.random.uniform(-0.5*model_rock['ly']+R,+0.5*model_rock['ly']-R,n) 
Coordz = np.random.uniform(-0.5*model_rock['lz']+R,+0.5*model_rock['lz']-R,n)
dec_rand = np.random.uniform(dec_min,dec_max,n)
inc_rand = np.random.uniform(inc_min,inc_max,n)

#### Random magnetization direction

In [14]:
mag_sph= 0.0000000000000001
mag_rand = []
for i in range(n):
    mag_rand.append(ang2vec(mag_sph,inc_rand[i],dec_rand[i]))

In [15]:
model_rock['spheres'] = []
for i in range(n):
    model_rock['spheres'].append(Sphere(Coordx[i], Coordy[i], Coordz[i], R , {'magnetization': mag_rand[i]}))

#### Generating .pickle file

In [16]:
now = datetime.datetime.utcnow().strftime('%d %B %Y %H:%M:%S UTC')
model_rock['metadata'] = 'Generated by {name} on {date}'.format(date=now, name=notebook_name)

In [17]:
file_name = 'data/model_rock.pickle'
with open(file_name, 'w') as f:
    pickle.dump(model_rock, f)
    
saved_files.append(file_name)

#### Saved files

In [18]:
with open('reports/report_%s.md' % notebook_name[:st.index(notebook_name, '.')], 'w') as q:
    q.write('# Saved files \n')
    now = datetime.datetime.utcnow().strftime('%d %B %Y %H:%M:%S UTC')
    header = 'Generated by {name} on {date}'.format(date=now, name=notebook_name)
    q.write('\n\n'+header+'\n\n')
    for i, sf in enumerate(saved_files):
        print '%d  %s' % (i+1,sf)
        q.write('*  `%s` \n' % (sf))

1  data/model_rock.pickle
